In [ ]:
shp = '/mnt/d/osm2lulc01/guavsforest.gdb/uatlas/forest_and_evu'

colcol = 'code_2018'

valcol = {
    'garea' : [
        [0, 1],
        [1, 3],
        [3, 5],
        [5, 10],
        [10, 15],
        [15, 30],
        [30, 100],
        [100, 6000]
    ],
    'direcoes'  : None,
    'areaurban' : [
        [0, 1],
        [1, 2],
        [2, 5],
        [5, 10],
        [10, 20],
        [20, 65]
    ],
    'npnts_evu' : [
        [0, 10],
        [10, 30],
        [30, 50],
        [50, 100],
        [100, 1873]
    ],
    'lenlines_evu' : [
        [0, 1],
        [1, 3],
        [3, 5],
        [5, 10],
        [10, 14]
    ],
    'poly_evu' : [
        [0, 10],
        [10, 30],
        [30, 50],
        [50, 70],
        [70, 101]
    ],
    'npnts_for' : [
        [0, 5],
        [5, 10],
        [10, 20],
        [20, 30],
        [30, 40]
    ],
    'lenlines_for' : [
        [0, 1],
        [1, 5],
        [5, 10],
        [10, 50],
        [50, 100],
        [100, 222]
    ],
    'denspop' : [
        [0, 100],
        [100, 250],
        [250, 500],
        [500, 1000],
        [1000, 2000],
        [2000, 5000],
        [5000, 10000],
        [10000, 42000]
    ],
    'pedificado_evu' : [
        [0, 0.5],
        [0.5, 1],
        [1, 10],
        [10, 250]
    ]
}

out_tbl = '/mnt/d/osm2lulc01/var_frequencies2.xlsx'

In [ ]:
import numpy as np
import pandas as pd

from glass.rd.shp import shp_to_obj
from glass.wt import obj_to_tbl

In [ ]:
gdf = shp_to_obj(shp)

In [ ]:
classes = gdf[colcol].unique()

In [ ]:
# Give classes to each record

_df = gdf.copy(deep=True)

_df.drop([c for c in _df.columns.values if c != colcol], axis=1, inplace=True)

freq, sheets = [], []

for c in valcol:
    if valcol[c]:
        for i in range(len(valcol[c])):
            if not i:
                _df[c] = np.where(
                    gdf[c] <= valcol[c][i][1],
                    #str(valcol[c][i][0]) + ' - ' + str(valcol[c][i][1]), ''
                    i + 1, 0
                )

                continue

            _df[c] = np.where(
                (gdf[c] > valcol[c][i][0]) & (gdf[c] <= valcol[c][i][1]),
                #str(valcol[c][i][0]) + ' - ' + str(valcol[c][i][1]), _df[c]
                i + 1, _df[c]
            )
    else:
        _df[c] = gdf[c]

    cls_dfs = []
    for cls in classes:
        cdf = _df[_df[colcol] == cls]

        fq_df = pd.DataFrame({
            f'fabs_{cls}' : cdf.groupby([c])[c].agg('count')
        }).reset_index()

        fq_df[f'aabs_{cls}'] = fq_df[f'fabs_{cls}'].cumsum()

        fq_df[f'frel_{cls}'] = fq_df[f'fabs_{cls}'] / fq_df[f'fabs_{cls}'].sum() * 100

        fq_df[f'arel_{cls}'] = fq_df[f'frel_{cls}'].cumsum()

        cls_dfs.append(fq_df)

    for i in range(len(cls_dfs)):
        if not i:
            fdf = cls_dfs[i]
            continue

        cls_dfs[i].rename(columns={c : f'{c}_y'}, inplace=True)

        fdf = fdf.merge(
            cls_dfs[i], how='outer',
            left_on=c, right_on=f'{c}_y'
        )

        fdf[c] = fdf[c].fillna(value=fdf[f'{c}_y'])

        fdf.drop(f'{c}_y', axis=1, inplace=True)

        fdf = fdf.fillna(0)

    freq.append(fdf)
    sheets.append(c)

In [ ]:
for e in range(len(freq)):
    if not valcol[sheets[e]]:
        continue

    freq[e]['label'] = ''
    
    for r in range(len(valcol[sheets[e]])):
        freq[e]['label'] = np.where(
            freq[e][sheets[e]] == r + 1,
            f'{str(valcol[sheets[e]][r][0])} <-> {str(valcol[sheets[e]][r][1])}',
            freq[e]['label'] 
        )

In [ ]:
obj_to_tbl(freq, out_tbl, sheetsName=sheets)